In [66]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import urllib.response
import json
from pandas.io.json import json_normalize
import os
import csv
import time
import concurrent.futures
import datetime as dt
from datetime import date, datetime, timezone
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression


# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "Weather_File/weather_API_data.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [67]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

634

In [68]:
#function to build api requests,  idiom represents reference to pull information
def pull_requests(city):
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = base_url + "appid="+ weather_api_key+"&units=imperial"+"&q="+city
    return complete_url
pull_requests(city)

'http://api.openweathermap.org/data/2.5/weather?appid=48cd2f4c6e1cc4ac15520e0f752a9eb5&units=imperial&q=cape town'

In [70]:
#function to get data and log as text in data folder
def pull_data(city, clause, output_data_text, counter = 0, set_num= 1 ):
    url = pull_requests(city)
    # name text document in data file
    printlog = open(f'data/{output_data_text}.txt', 'w')
    try:
        response = requests.get(url)
        response.status_code
        call = response.json()
        if counter >= 50:
            counter=1
            set_num+=1
        
        else:
            counter+=1
        # 'Response' object is not subscriptable use variable call to get city id
        city_id = int(call["id"])
        print(f"Currently Processing record number {counter} of Set {set_num} | {city} with City ID: {city_id}")
        printlog.write(f"Currently Processing record number {counter} of Set {set_num} | {city} with City ID: {city_id}")

    # Add string to comment on download issue, perhaps url has a compenent not found clause
    except requests.exceptions.HTTPError:
        print('Error occured while downloading data ')
        pass
    except urllib.error.URLError:
        print('Error occured while downloading data... ' + clause) 
        printlog.write('Error occured while downloading data... ' + clause)
        pass
    
    # name text document in data file
    JSON_data = response.text
    data = json.loads(JSON_data)
    return data
pull_data(city, "City not found .... ", "weather_data_log") 


Currently Processing record number 1 of Set 1 | cape town with City ID: 3369157


{'coord': {'lon': 18.42, 'lat': -33.93},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 55.08,
  'feels_like': 51.46,
  'temp_min': 55,
  'temp_max': 55.4,
  'pressure': 1025,
  'humidity': 71},
 'visibility': 10000,
 'wind': {'speed': 4.7, 'deg': 220},
 'clouds': {'all': 75},
 'dt': 1597289978,
 'sys': {'type': 1,
  'id': 1899,
  'country': 'ZA',
  'sunrise': 1597296435,
  'sunset': 1597335313},
 'timezone': 7200,
 'id': 3369157,
 'name': 'Cape Town',
 'cod': 200}

In [34]:
# # Make dataframe and manipulate columns!
# def json_to_df(data, columns):
#     df = pd.DataFrame.from_dict(json_normalize(data), orient='columns')
#     new_df = df[columns]
#     new_df.insert(0, 'TimeStamp', pd.datetime.now().replace(microsecond=0))
#     if not os.path.isfile('weather.csv'):
#         return new_df.to_csv('weather.csv', header='column_names', index=False)
#     else:
#         return new_df.to_csv('weather.csv', mode='a', header=False, index=False)  

In [62]:
#creating a dictionary of cities and city codes(based on the CSV file downloaded from a given URL
def create_dict(URL):
    with requests.Session() as s:
        dict = {}
        download = s.get(URL)
        decoded_content = download.content.decode('utf-8')
        cs = csv.reader(decoded_content.splitlines(), delimiter=',')
        next(cs, None)
        my_list = list(cs)
        for row in my_list:
            dict[row[0]] = row[1]
        return dict
    #request is limited

In [63]:
# String_url = "jdbc:postgresql://localhost/postgres";
# Properties props = new Properties();
# props.setProperty("user","postgres");
# props.setProperty("password","postgres");
# props.setProperty("ssl","true");
# Connection_conn = DriverManager.getConnection(url, props);

# String_url = "jdbc:postgresql://localhost/test?user=fred&password=secret&ssl=true";
# Connection_conn = DriverManager.getConnection(url);

In [71]:
# store =  "jdbc:postgresql://localhost/postgres?user:postgrespassword:postgres&ssl=true"
# columns = [ "['dt']", "['name']", "['sys']['country']", 
#            "['coord']['lat']", 
#            "['coord']['lon']", "['clouds']['all']", 
#            "['main']['humidity']", "['main']['temp_max']", 
#            "['wind']['speed']"]
# #Writing results to CSV
# def weather_api(store):
#     dict = create_dict(store)
#     for city, code in dict.items():
#         data = get_weather_data(city, code)
#         json_to_df(data, columns)
# #sqlalchemy 